```i:O want:O to:O fly:O from:O baltimore:B-fromloc.city_name to:O dallas:B-toloc.city_name round:B-round_trip trip:I-round_trip <=> atis_flight```

i want to fly from <ins>**baltimore**</ins> to <ins>**dallas**</ins> <ins>**round trip**</ins> 
* B-fromloc.city_name
* B-toloc.city_name
* B-round_trip 


In [1]:
!cat  data/train_dev | head -n 5 

i:O want:O to:O fly:O from:O baltimore:B-fromloc.city_name to:O dallas:B-toloc.city_name round:B-round_trip trip:I-round_trip <=> atis_flight
round:B-round_trip trip:I-round_trip fares:O from:O baltimore:B-fromloc.city_name to:O philadelphia:B-toloc.city_name less:B-cost_relative than:O 1000:B-fare_amount dollars:I-fare_amount round:B-round_trip trip:I-round_trip fares:O from:O denver:B-fromloc.city_name to:O philadelphia:B-toloc.city_name less:B-cost_relative than:O 1000:B-fare_amount dollars:I-fare_amount round:B-round_trip trip:I-round_trip fares:O from:O pittsburgh:B-fromloc.city_name to:O philadelphia:B-toloc.city_name less:B-cost_relative than:O 1000:B-fare_amount dollars:I-fare_amount <=> atis_airfare
show:O me:O the:O flights:O arriving:O on:O baltimore:B-toloc.city_name on:O june:B-arrive_date.month_name fourteenth:B-arrive_date.day_number <=> atis_flight
what:O are:O the:O flights:O which:O depart:O from:O san:B-fromloc.city_name francisco:I-fromloc.city_name fly:O to:O washi

In [2]:
!python train.py 

Number of training samples:  4978
Number of test samples:  893
Number of words:  754
Number of intent labels:  18
Number of slot labels 130
Slot(
  (enc): slot_enc(
    (embedding): Embedding(754, 300)
    (lstm): LSTM(300, 200, num_layers=2, batch_first=True, bidirectional=True)
  )
  (dec): slot_dec(
    (lstm): LSTM(1000, 200)
    (fc): Linear(in_features=200, out_features=130, bias=True)
  )
)
Intent(
  (enc): intent_enc(
    (embedding): Embedding(754, 300)
    (lstm): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (dec): intent_dec(
    (lstm): LSTM(800, 200, batch_first=True)
    (fc): Linear(in_features=200, out_features=18, bias=True)
  )
)
^C
Traceback (most recent call last):
  File "train.py", line 66, in <module>
    intent_loss.backward()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/tensor.py", line 195, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
model_intent = torch.load('model_intent_best.ckpt')
model_slot = torch.load('model_slot_best.ckpt')






In [4]:
from data2index_ver2 import train_data, test_data, index2slot_dict
from make_dict import inv_word_dict

print(len(test_data)) 

Number of training samples:  4978
Number of test samples:  893
Number of words:  754
Number of intent labels:  18
Number of slot labels 130
893


In [11]:
import utils
from utils import get_chunks
from config import device
import config as cfg
from model import *
TP=0
FP=0
TN=0
FN=0
correct_num = 0 
total_test = len(test_data)


for batch_index, data_test in enumerate(utils.get_batch(test_data, batch_size=1)):
    sentence_test, real_len_test, slot_label_test, intent_label_test = data_test
    # print(sentence[0].shape, real_len.shape, slot_label.shape)

    x_test = torch.tensor(sentence_test).to(device)

    mask_test = utils.make_mask(real_len_test, batch=1).to(device)
        # Slot model generate hs_test and intent model generate hi_test
    hs_test = model_slot.enc(x_test)
    hi_test = model_intent.enc(x_test)

        # Slot
    slot_logits_test = model_slot.dec(hs_test, hi_test)
    log_slot_logits_test = utils.masked_log_softmax(slot_logits_test, mask_test, dim=-1)
    slot_pred_test = torch.argmax(log_slot_logits_test, dim=-1)
        # Intent
    intent_logits_test = model_intent.dec(hi_test, hs_test, real_len_test)
    log_intent_logits_test = F.log_softmax(intent_logits_test, dim=-1)
    res_test = torch.argmax(log_intent_logits_test, dim=-1)
        

    if res_test.item() == intent_label_test[0]:
        correct_num += 1

        
    slot_pred_test = slot_pred_test[0][:real_len_test[0]]
    slot_label_test = slot_label_test[0][:real_len_test[0]]

    slot_pred_test = [int(item) for item in slot_pred_test]
    slot_label_test = [int(item) for item in slot_label_test]

    slot_pred_test = [index2slot_dict[item] for item in slot_pred_test]
    slot_label_test = [index2slot_dict[item] for item in slot_label_test]
    slot_results = [] 

    print(len(slot_pred_test), len(sentence_test)) 
    for i in range(0, len(sentence_test[0])): 
        tok = sentence_test[0][i]
        if not tok==1:
            slot_results.append(inv_word_dict[tok]+":"+slot_pred_test[i])
        else:
            break
    print(slot_results)        
    

    pred_chunks = get_chunks(['O'] + slot_pred_test + ['O'])
    label_chunks = get_chunks(['O'] + slot_label_test + ['O'])
    for pred_chunk in pred_chunks:
        if pred_chunk in label_chunks:
            TP += 1
        else:
            FP += 1
    for label_chunk in label_chunks:
        if label_chunk not in pred_chunks:
            FN += 1

    F1_score = 100.0*2*TP/(2*TP+FN+FP)

print('*'*20)
print('Intent Val Acc: {:.4f} \t Slot F1 score: {:.4f}'.format(100.0*correct_num/total_test, F1_score))
print('*'*20)
    
# print('Best Intent Acc: {:.4f} at Epoch: [{}]'.format(100.0*best_correct_num/total_test, best_epoch+1))
# print('Best F1 score: {:.4f} at Epoch: [{}]'.format(best_F1_score, best_epoch_slot+1))

15 1
['find:O', 'me:O', 'a:O', 'flight:O', 'from:O', 'cincinnati:B-fromloc.city_name', 'to:O', 'any:O', 'airport:O', 'in:O', 'the:O', 'new:B-fromloc.city_name', 'york:I-fromloc.city_name', 'city:I-fromloc.city_name', 'area:O']
7 1
['please:O', 'list:O', 'flights:O', 'from:O', 'milwaukee:B-fromloc.city_name', 'to:O', 'philadelphia:B-toloc.city_name']
4 1
['what:O', 'airline:O', 'is:O', 'as:B-airline_code']
10 1
['find:O', 'american:B-airline_name', 'flight:O', 'from:O', 'newark:B-fromloc.city_name', 'to:O', 'nashville:B-toloc.city_name', 'around:B-depart_time.time_relative', 'DIGITDIGITDIGIT:B-depart_time.time', 'pm:I-depart_time.time']
10 1
['which:O', 'flights:O', 'go:O', 'from:O', 'new:B-fromloc.city_name', 'york:I-fromloc.city_name', 'to:O', 'miami:B-toloc.city_name', 'and:O', 'back:O']
13 1
['new:B-fromloc.city_name', 'york:I-fromloc.city_name', 'city:I-fromloc.city_name', 'to:O', 'las:B-toloc.city_name', 'vegas:I-toloc.city_name', 'and:O', 'memphis:B-fromloc.city_name', 'to:O', 'l

6 1
['tell:O', 'me:O', 'about:O', 'the:O', 'm80:B-aircraft_code', 'aircraft:O']
9 1
['how:O', 'many:O', 'canadian:B-airline_name', 'airlines:I-airline_name', 'international:I-airline_name', 'flights:O', 'use:O', 'aircraft:O', 'DIGITDIGITDIGIT:B-aircraft_code']
11 1
['list:O', 'the:O', 'flights:O', 'from:O', 'westchester:B-fromloc.city_name', 'county:I-fromloc.city_name', 'to:O', 'denver:B-toloc.city_name', 'on:O', 'june:B-depart_date.month_name', 'seventh:B-depart_date.day_number']
10 1
['list:O', 'flights:O', 'from:O', 'salt:B-fromloc.city_name', 'lake:I-fromloc.city_name', 'city:I-fromloc.city_name', 'to:O', 'phoenix:B-toloc.city_name', 'arizona:B-toloc.state_name', 'tuesday:B-depart_date.day_name']
8 1
['find:O', 'a:O', 'flight:O', 'from:O', 'memphis:B-fromloc.city_name', 'to:O', 'tacoma:B-toloc.city_name', 'dinner:B-meal_description']
8 1
['i:O', 'need:O', 'a:O', 'flight:O', 'from:O', 'memphis:B-fromloc.city_name', 'to:O', 'seattle:B-toloc.city_name']
11 1
['which:O', 'different:O'

11 1
['tell:O', 'me:O', 'all:O', 'the:O', 'airports:O', 'in:O', 'the:O', 'new:B-city_name', 'york:I-city_name', 'city:I-city_name', 'area:I-city_name']
13 1
['list:O', 'flights:O', 'from:O', 'oakland:B-fromloc.city_name', 'to:O', 'salt:B-toloc.city_name', 'lake:I-toloc.city_name', 'city:I-toloc.city_name', 'before:B-depart_time.time_relative', 'DIGIT:B-depart_time.time', 'am:I-depart_time.time', 'thursday:B-depart_date.day_name', 'morning:B-depart_time.period_of_day']
4 1
['what:O', 'airlines:O', 'serve:O', 'denver:B-city_name']
12 1
['show:O', 'me:O', 'the:O', 'first:B-class_type', 'class:I-class_type', 'and:O', 'coach:B-class_type', 'flights:O', 'from:O', 'jfk:B-fromloc.airport_code', 'to:O', 'miami:B-toloc.city_name']
9 1
['what:O', 'flights:O', 'arrive:O', 'at:O', 'love:B-toloc.airport_name', 'field:I-toloc.airport_name', 'on:O', 'june:B-arrive_date.month_name', 'sixth:B-arrive_date.day_number']
9 1
['i:O', 'need:O', 'a:O', 'flight:O', 'from:O', 'toronto:B-fromloc.city_name', 'to:O

8 1
['what:O', 'is:O', 'the:O', 'seating:O', 'capacity:O', 'for:O', 'delta:B-airline_name', 'UNK:I-airline_name']
5 1
['what:O', 'is:O', 'fare:O', 'code:O', 'f:B-fare_basis_code']
13 1
['i:O', 'would:O', 'like:O', 'an:O', 'early:B-depart_time.period_of_day', 'morning:B-depart_time.period_of_day', 'flight:B-flight_time', 'today:B-depart_date.today_relative', 'from:O', 'los:B-fromloc.city_name', 'angeles:I-fromloc.city_name', 'to:O', 'charlotte:B-toloc.city_name']
5 1
['tampa:B-fromloc.city_name', 'to:O', 'charlotte:B-toloc.city_name', 'sunday:B-depart_date.day_name', 'morning:B-depart_time.period_of_day']
9 1
['please:O', 'list:O', 'the:O', 'flights:O', 'from:O', 'los:B-fromloc.city_name', 'angeles:I-fromloc.city_name', 'to:O', 'charlotte:B-toloc.city_name']
2 1
['list:O', 'airports:O']
10 1
['kansas:B-fromloc.city_name', 'city:I-fromloc.city_name', 'to:O', 'cleveland:B-toloc.city_name', 'flight:O', 'arrive:O', 'wednesday:B-arrive_date.day_name', 'before:B-arrive_time.time_relative', 'D

5 1
['newark:B-fromloc.city_name', 'to:O', 'tampa:B-toloc.city_name', 'on:O', 'friday:B-depart_date.day_name']
6 1
['what:O', 'airlines:O', 'are:O', 'departing:O', 'from:O', 'baltimore:B-fromloc.city_name']
6 1
['get:O', 'fares:O', 'from:O', 'washington:B-fromloc.city_name', 'to:O', 'montreal:B-toloc.city_name']
17 1
['please:O', 'find:O', 'a:O', 'flight:O', 'that:O', 'goes:O', 'from:O', 'kansas:B-fromloc.city_name', 'city:I-fromloc.city_name', 'to:O', 'newark:B-toloc.city_name', 'to:O', 'orlando:B-toloc.city_name', 'back:O', 'to:O', 'kansas:B-toloc.city_name', 'city:I-toloc.city_name']
9 1
['how:O', 'about:O', 'from:O', 'dc:B-fromloc.state_code', 'to:O', 'oakland:B-toloc.city_name', 'on:O', 'wednesday:B-depart_date.day_name', 'morning:B-depart_time.period_of_day']
13 1
['what:O', 'ground:O', 'transportation:O', 'is:O', 'available:O', 'from:O', 'dallas:B-fromloc.airport_name', 'fort:I-fromloc.airport_name', 'worth:I-fromloc.airport_name', 'airport:I-fromloc.airport_name', 'to:O', 'down

...

7 1
['list:O', 'the:O', 'alaska:B-airline_name', 'airline:I-airline_name', 'flights:O', 'from:O', 'burbank:B-fromloc.city_name']
13 1
['i:O', 'need:O', 'the:O', 'fares:O', 'on:O', 'flights:O', 'from:O', 'washington:B-fromloc.city_name', 'to:O', 'toronto:B-toloc.city_name', 'on:O', 'a:O', 'saturday:B-depart_date.day_name']
3 1
['list:O', 'california:B-airport_code', 'airports:O']
4 1
['what:O', 'does:O', 'ua:B-airline_code', 'mean:O']
6 1
['what:O', 'does:O', 'fare:O', 'code:O', 'h:B-fare_basis_code', 'mean:O']
11 1
['what:O', 'ground:O', 'transportation:O', 'is:O', 'available:O', 'between:O', 'phoenix:B-fromloc.airport_name', 'airport:I-fromloc.airport_name', 'and:O', 'downtown:O', 'phoenix:B-toloc.city_name']
13 1
['give:O', 'me:O', 'the:O', 'flights:O', 'from:O', 'phoenix:B-fromloc.city_name', 'to:O', 'milwaukee:B-toloc.city_name', 'on:O', 'wednesday:B-depart_date.day_name', 'on:O', 'american:B-airline_name', 'airlines:I-airline_name']
11 1
['what:O', 'are:O', 'the:O', 'fares:O', 'fo

6 1
['flights:O', 'from:O', 'san:B-fromloc.city_name', 'diego:I-fromloc.city_name', 'to:O', 'seattle:B-toloc.city_name']
10 1
['what:O', 'is:O', 'the:O', 'shortest:B-flight_mod', 'flight:O', 'from:O', 'milwaukee:B-fromloc.city_name', 'to:O', 'long:B-toloc.city_name', 'beach:I-toloc.city_name']
12 1
['list:O', 'us:B-airline_name', 'air:I-airline_name', 'flights:O', 'leaving:O', 'saturday:B-depart_date.day_name', 'from:O', 'charlotte:B-fromloc.city_name', 'airport:I-fromloc.airport_name', 'at:O', 'DIGIT:B-depart_time.time', 'pm:I-depart_time.time']
11 1
['list:O', 'the:O', 'flights:O', 'from:O', 'indianapolis:B-fromloc.city_name', 'to:O', 'memphis:B-toloc.city_name', 'that:O', 'leave:O', 'before:B-depart_time.time_relative', 'noon:B-depart_time.time']
16 1
['give:O', 'me:O', 'the:O', 'saturday:B-depart_date.day_name', 'morning:B-depart_time.period_of_day', 'flights:O', 'from:O', 'chicago:B-fromloc.city_name', 'to:O', 'st.:B-toloc.city_name', 'paul:I-toloc.city_name', 'on:O', 'continental

10 1
['i:O', 'want:O', 'to:O', 'fly:O', 'from:O', 'washington:B-fromloc.city_name', 'dc:B-fromloc.state_code', 'to:O', 'phoenix:B-toloc.city_name', 'arizona:B-toloc.state_name']
12 1
['show:O', 'me:O', 'flight:O', 'us:B-airline_code', 'DIGITDIGITDIGITDIGIT:B-flight_number', 'on:O', 'monday:B-depart_date.day_name', 'from:O', 'charlotte:B-fromloc.city_name', 'to:O', 'minneapolis:B-toloc.city_name', 'please:O']
11 1
["i'd:O", 'like:O', 'to:O', 'fly:O', 'from:O', 'miami:B-fromloc.city_name', 'to:O', 'chicago:B-toloc.city_name', 'on:O', 'american:B-airline_name', 'airlines:I-airline_name']
7 1
['please:O', 'list:O', 'flights:O', 'from:O', 'orlando:B-fromloc.city_name', 'to:O', 'philadelphia:B-toloc.city_name']
12 1
['cheapest:B-cost_relative', 'fare:O', 'round:B-round_trip', 'trip:I-round_trip', 'from:O', 'indianapolis:B-fromloc.city_name', 'to:O', 'orlando:B-toloc.city_name', 'on:O', 'december:B-depart_date.month_name', 'twenty:B-depart_date.day_number', 'seventh:I-depart_date.day_number']

16 1
['which:O', 'flights:O', 'are:O', 'available:O', 'from:O', 'tacoma:B-fromloc.city_name', 'to:O', 'san:B-toloc.city_name', 'jose:I-toloc.city_name', 'in:O', 'the:O', 'morning:B-depart_time.period_of_day', 'on:O', 'april:B-depart_date.month_name', 'twenty:B-depart_date.day_number', 'second:I-depart_date.day_number']
10 1
['list:O', 'the:O', 'flights:O', 'from:O', 'cincinnati:B-fromloc.city_name', 'to:O', 'denver:B-toloc.city_name', 'on:O', 'june:B-depart_date.month_name', 'sixth:B-depart_date.day_number']
9 1
['what:O', 'are:O', 'the:O', 'fares:O', 'for:O', 'ground:O', 'transportation:O', 'in:O', 'denver:B-city_name']
12 1
['what:O', 'meals:B-meal', 'are:O', 'served:O', 'on:O', 'american:B-airline_name', 'flight:O', 'DIGITDIGITDIGIT:B-flight_number', 'from:O', 'tampa:B-fromloc.city_name', 'to:O', 'milwaukee:B-toloc.city_name']
15 1
['find:O', 'me:O', 'all:O', 'the:O', 'round:B-round_trip', 'trip:I-round_trip', 'flights:O', 'from:O', 'burbank:B-fromloc.city_name', 'to:O', 'milwaukee:

12 1
['does:O', 'tacoma:B-airport_name', 'airport:I-airport_name', 'offer:O', 'transportation:O', 'from:O', 'the:O', 'airport:O', 'to:O', 'the:O', 'downtown:O', 'area:O']
8 1
['what:O', 'are:O', 'the:O', 'flights:O', 'from:O', 'milwaukee:B-fromloc.city_name', 'to:O', 'seattle:B-toloc.city_name']
7 1
['how:O', 'many:O', 'northwest:B-airline_name', 'flights:O', 'leave:O', 'st.:B-fromloc.city_name', 'paul:I-fromloc.city_name']
4 1
['what:O', 'airline:O', 'is:O', 'hp:B-airline_code']
14 1
['please:O', 'find:O', 'a:O', 'flight:O', 'from:O', 'detroit:B-fromloc.city_name', 'michigan:B-fromloc.state_name', 'to:O', 'st.:B-toloc.city_name', 'petersburg:I-toloc.city_name', 'arriving:O', 'before:B-arrive_time.time_relative', 'DIGITDIGIT:B-arrive_time.time', 'pm:I-arrive_time.time']
10 1
['now:O', 'show:O', 'me:O', 'one:B-round_trip', 'way:I-round_trip', 'flights:O', 'from:O', 'houston:B-fromloc.city_name', 'to:O', 'boston:B-toloc.city_name']
12 1
['i:O', 'would:O', 'like:O', 'information:O', 'on:O